# Web-Scraping

## Objectives

- Parse HTML elements in webpages
- Use requests and BeautifulSoup to get and process webpage contents
- Use ethics when scraping websites

Hopefully, you can get all the data you need easily and accessibly, and don't need to scour the web to find a source that will let you do your analysis. 

We'd all prefer one of these:

<img src="images/other_options.png" alt="image showcasing a downloadable csv, database connection, or API, but we're not always so lucky. not sure of image source, took from materials provided by another instructor" width=650>

But we're not always so lucky! Sometimes we need data that's less accessible.

Enter...

<img alt="beautiful soup logo" src="images/bs.png" width=500>

> "You didn't write that awful page. You're just trying to get some data out of it. Beautiful Soup is here to help. Since 2004, it's been saving programmers hours or days of work on quick-turnaround screen scraping projects."

- From the Beautiful Soup [documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#)

## The components of a web page

When we visit a web page, our web browser makes a GET request to a web server. The server then sends back files that tell our browser how to render the page for us. The files fall into a few main types:

- HTML — contain the main content of the page.
- CSS — add styling to make the page look nicer.
- JS — Javascript files add interactivity to web pages.
- Images — image formats, such as JPG and PNG allow web pages to show pictures.

After our browser receives all the files, it renders the page and displays it to us. There’s a lot that happens behind the scenes to render a page nicely, but we don’t need to worry about most of it when we’re web scraping.

### HTML

HyperText Markup Language (HTML) is a language that web pages are created in. HTML isn’t a programming language, like Python — instead, it’s a markup language that tells a browser how to layout content. Let’s take a quick tour through HTML so we know enough to scrape effectively.

HTML consists of elements called tags.

Tags have commonly used names that depend on their position in relation to other tags:

- **child** — a child is a tag inside another tag. 
- **parent** — a parent is the tag another tag is inside. 
- **sibling** — a sibiling is a tag that is nested inside the same parent as another tag. 

Here's som example HTML - which tags are parents? children? siblings?

~~~html
<html>
  <head></head>
  <body>
    <p>
      Here's a paragraph of text!
      <a href="https://www.dataquest.io">Learn Data Science Online</a>
    </p>
    <p>
      Here's a second paragraph of text!
      <a href="https://www.python.org">Python</a>        
    </p>
  </body>
</html>
~~~

## Grabbing Movie Data

We might think about grabbing more movie data, as we gear up for our Phase 1 project which uses movie data. 

If we go to [IMDB](https://www.imdb.com/), their only API content seems expensive, and their advanced search results in tabular data that seems _extremely_ scrapable.

**BUT** 

Enter - [conditions of use pages](https://www.imdb.com/conditions) ... and ethics!

> "**Robots and Screen Scraping:** You may not use data mining, robots, screen scraping, or similar data gathering and extraction tools on this site, except with our express written consent as noted below."


### Ethical Concerns

- Terms of Service (includes the conditions of use shown above)
- Denial of Service Attacks
- Confidentiality

[This article](https://oxylabs.io/blog/is-web-scraping-legal) discusses legal issues related to web scraping.

Key points: 

- Don't log into a site and then scrape what's only available after logging in - then, you're likely violating the Terms of Service (you can always check to see if that's actually covered in the ToS)
- Don't scrape copyrighted data 

**Let's Discuss**

- Do people scrape sites they shouldn't? Sure, all the time. But am I going to tell you to ignore conditions/terms of use? Absolutely not. Make good choices.

_We are not lawyers - this does not constitute legal advice._

Instead, let's scrape Wikipedia for movie data - Wikipedia has a very accessible Creative Commons license for use!

Let's explore a few [years in film](https://en.wikipedia.org/wiki/Table_of_years_in_film).

## Task: Grab the top 10 highest-grossing films for each year, 2000-2020

### Imports

Our goal is to collect data into a Pandas dataframe. Plus we're still working with websites, so we'll still need the requests library.

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup # note this odd import statement structure

In [ ]:
# we may also need lxml - https://lxml.de/index.html
# helps process html or xml in python
# !pip install lxml

Test case - [the year 2000](https://en.wikipedia.org/wiki/2000_in_film).

In [ ]:
# Get the response from the website, using requests
resp = None

In [ ]:
# Let's check out the text attribute of that response...
resp.text
# (ew)

In [ ]:
# And now... beautiful soup! Let's soup-ify that text attribute
soup = None

In [ ]:
# Can use a prettify function to pretty print
print(soup.prettify())

In [ ]:
# Now we need to find the table we want in the soup - use .find()
# Can pass a dictionary in the attributes argument
table = None

In [ ]:
# Explore that result


In [ ]:
# Check out the first real row in the table
# Can use get_text as a method, then use string methods!


In [ ]:
# Check out the last row


In [ ]:
# Can make into a dataframe like we did with an API result
# First need to define the columns
columns = table.find_all('tr')[0].get_text().split('\n')[1:5]
columns

In [ ]:
# Now let's make a list of rows for the data
row_list = []
for row in table.find_all('tr')[1:]:
    row_list.append(row.get_text().split('\n\n'))
    
row_list

In [ ]:
df = pd.concat([pd.DataFrame([i], columns = columns) for i in row_list], ignore_index=True)
df

In [ ]:
# Clean that up... Using an applymap!
df.applymap(lambda x: x.replace("\n", ""))

**But wait...** there's a shortcut (thanks pandas)

In [ ]:
# Check out the read_html method
# Note - pandas likes the prettify objects better
pd.read_html(table.prettify())[0]

In [ ]:
df = pd.read_html(table.prettify())[0]
df

In [ ]:
# Can add a column saying which year this ranking is from
df['Year'] = 2000

In [ ]:
df

### Now Loop It!

In [ ]:
# My preference - create a list of dataframes, then concat afterwards
# Are there other ways to create one big df from this? OF COURSE!

list_of_dfs = []

for year in range(2000, 2021):
    url = f"https://en.wikipedia.org/wiki/{year}_in_film"
    resp = requests.get(url).text
    soup = BeautifulSoup(resp)
    table = soup.find('table', {'class':"wikitable sortable"})
    df = pd.read_html(table.prettify())[0]
    df['Year'] = year
    list_of_dfs.append(df)
    # Only 20 things... not going to worry about using time to pause requests

In [ ]:
# Check out the last one
list_of_dfs[20]

In [ ]:
# Now to concat...
full_df = pd.concat([df for df in list_of_dfs], ignore_index=True)
full_df

Let's practice some data cleaning on the Worldwide Gross column:

In [ ]:
# Can see a hint of some gross data in the Gross column


In [ ]:
# Let's check out that complicated example


In [ ]:
# Test out how to clean it...


In [ ]:
# Now let's do it on the whole column


In [ ]:
# Check our work


In [ ]:
# Can also remove commas and dollar signs, and make the column integers!


In [ ]:
# Sanity check


## Discussion Time!

What else could we do with webscraping? Any project ideas pop into mind? Any useful things on that page we could also use to grab more data? Let's discuss!

- Had URLs in these results - could grab even more data on each movie using those
- Can loop through any kind of repeatable URL, provided you figure out the pattern!
- The possibilities are endless... (but don't forget to check the terms of use or copyright!)
